In [4]:
import torch
from torch.nn import *
import torch.nn.functional as F
from pandas_datareader import data as web
import os
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from diebold_mariano import diebold_mariano_test

In [5]:
from utils import *

DEVICE:  cuda


In [6]:
T=30

In [7]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib

# Set consistent style for publication-quality plots
sns.set(style="whitegrid", context="talk", palette="colorblind")
matplotlib.rcParams.update({
    "axes.titlesize": 18,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "legend.frameon": True,
    "axes.titleweight": "bold"
})

def customize_ax(ax, title=None, xlabel=None, ylabel=None):
    if title:
        ax.set_title(title, fontsize=20, fontweight="bold")
    if xlabel:
        ax.set_xlabel(xlabel, fontsize=12)
    if ylabel:
        ax.set_ylabel(ylabel, fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(False)
    if ax.get_legend():
        ax.legend(fontsize=10, frameon=True)

# Insert your original loop code here afterward, using customize_ax() and matplotlib.rcParams updates
# on all relevant plots to ensure publication-quality formatting.


In [8]:
etfs = ['XLB','XLE','XLF','XLI','XLK','XLP','XLV','XLY','XLU']
metric2 = "SD_Squared_Returns"
metric3 = "SD_Prices"
metric4 = "IQR"
metrics = ["ATR",metric2,metric3,metric4]
# start = '2001-01-01'
# end = '2020-01-01'
# start_t = '2022-01-02'
# end_t = '2024-01-05'
date_list = [['2001-01-01', '2020-01-01', '2022-01-02', '2024-01-05'], ['1990-01-05', '2010-01-05', '2023-01-05', '2024-01-05'], ['2000-01-05', "2024-01-05", "2024-01-05", "2025-01-05"], ['2005-01-05', '2010-01-05', '2020-01-05', '2024-01-05']]
starts,ends = np.array([(f'200{i}-01-01',f'200{i+12}-01-01') for i in range(0,12,4)]).T
models = {}
lrs = [0]
h1 = 50
h2 = 50
hl1s = [20,50,70,100]
hl1s = [20,50,70,100]
models = ["LSTM","RNN"]
b_types = ['ATR' ,'STD']
lr = 0
feat_combos = [['Open','Close','High','Low','Volume','Return','Short_MA','Long_MA'],['Open','Close','High','Low','Volume'],['index','Open','Close','High','Low','Volume']]

In [ ]:
num=20000
for file in os.listdir('../output'):
    if "runtest" in file:
        num+=1
os.mkdir(f'../output/runtest_{num}')
for d,(start,end,start_t,end_t) in enumerate(date_list):
    os.mkdir(f'../output/runtest_{num}/date_config{d}')
    for model_type in models:
        os.mkdir(f'../output/runtest_{num}/date_config{d}/{model_type}')
        root = f'../output/runtest_{num}/date_config{d}/{model_type}'
        with open(os.path.join(root,'run_data.txt'),'w') as f:
            f.write(f'Start Training Date: {start} \n End Training Date: {end} \n Start Sim Date: {start_t} \n End Sim Date: {end_t} \n')
            f.write(f'\n Model Type: {model_type}')
            f.write(f'Look-back window: {T} days')

        for etf in etfs:
            os.mkdir(f'{root}/{etf}')
        for lr in lrs:
            results = pd.DataFrame({"ETF":[],"Metric":[],"Final_Value":[],"Passive_Potential":[],"Relative_MSE":[],"MAPE":[],"DM_stat":[],"DM_p":[]})
            money_df_std = pd.DataFrame()
            money_df_atr = pd.DataFrame()
            money_df_sd_prices = pd.DataFrame()
            money_df_iqr = pd.DataFrame()
            money_df_passive = pd.DataFrame()
            for j,etf in enumerate(etfs):
                #fig,ax = plt.subplots(1,len(metrics),figsize=(26,6))
                fig,ax = plt.subplots(2,2,figsize=(25,18))
                atr_preds = []
                for i,metric in enumerate(metrics):
                    scaler = StandardScaler()
                    scaler_x = StandardScaler()
                    df = get_cleaned_df(etf,start,end)
                    #print(df)
                    model,fig_tr,fig_te = get_trained_model(df,scaler, scaler_x, metric=metric,model_type=model_type)
                    fig_tr.savefig(os.path.join(root,etf,f'training_results_{metric}'))
                    fig_te.savefig(os.path.join(root,etf,f'testing_results_{metric}'))
                    df_test = get_cleaned_df(etf,start_t,end_t)
                    ini_cash = 10000
                    final_value, money, shares,passive_value,buy_points,sell_points,preds, t_money,p_money = backtest_strategy_mr(df_test,model,scaler,scaler_x,metric,sell_scale=2,buy_scale=2)
                    print(f"{etf} Final Value: {final_value}, Final Money: {money}, Final Shares: {shares}")
                    
                    real_met = np.array(df_test[metric].iloc[T:len(df_test)-1])
                    atr_met = np.array(df_test["ATR"].iloc[T:len(df_test)-1])
                    null_model_error = np.sum((real_met-real_met.mean())**2)
                    mse_error = 1/len(real_met) * np.sum((real_met-preds)**2)/null_model_error
                    mape_error = 1/len(real_met) * np.sum(abs(real_met-preds)/(real_met+1e-8))
                    if metric=="ATR": #atr should go first
                        money_df_atr[etf] = t_money
                        atr_preds = preds
                    elif metric=="SD_Squared_Returns":
                        money_df_std[etf] = t_money
                        money_df_passive[f"{etf}_Passive"] = p_money
                    elif metric == "SD_Prices":
                        money_df_sd_prices[etf] = t_money
                    elif metric == "IQR":
                        money_df_iqr[etf] = t_money

                    dm,p = diebold_mariano_test(atr_met,atr_preds,real_met,preds,loss="mae")
                    results.loc[len(results)] = [etf,metric,final_value,passive_value,mse_error,mape_error,dm,p]

                    sns.lineplot(preds,ax=ax[i//2][i%2],label=f"Prediction {metric}",color="Blue",linestyle = '--')
                    sns.lineplot(real_met,ax=ax[i//2][i%2],label = f"True {metric}",color="Orange")
                    sns.despine(ax=ax[i//2][i%2])
                    #for buy in buy_points:
                        #ax[i].axvline(buy,color="Green",alpha=.6)
                    #for sell in sell_points:
                        #ax[i].axvline(sell,color="Red",alpha=.6)
                    ax[i//2][i%2].set_title(f"{etf} - {metric} ",fontsize = 20)
                    ax[i//2][i%2].set_xlabel(f'Total MSE Error {mse_error:.6f} | Total MAPE Error: {mape_error:.3f}')
                    customize_ax(ax[i//2][i%2])
                    #plt.title(f"Performance of LSTM on {etf}")
                fig.supylabel("Volatility Metric")
                fig.supxlabel(f"Days after {start_t}")
                fig.tight_layout()
                fig.savefig(os.path.join(root,etf,'predictions_lr_{lr}.png'))
                fig_sim,ax_sim = plt.subplots(figsize=(10,6))
                sns.lineplot(x=range(0, len(df_test.index)-(T+1)), y=money_df_atr[etf], label="ATR",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_std[etf],label="SD_Squared_Returns",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_sd_prices[etf],label="SD_Prices",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_passive[f"{etf}_Passive"],label="Passive",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_iqr[etf],label="IQR",ax=ax_sim,palette = 'colorblind')
                sns.despine(ax=ax_sim)
                ax_sim.axhline(1000, label = "Initial",color="Black")
                ax_sim.set_xlabel(f"Days after {start_t}")
                ax_sim.set_ylabel(f"Value ($)")
                ax_sim.set_title(f"ETF Performance Over Time for {etf}")
                customize_ax(ax_sim)
                ax_sim.legend(
                    loc='upper left',              # anchor the legend’s corner
                    bbox_to_anchor=(1.02, 1.0),     # (x, y) in axes fraction coordinates
                    borderaxespad=0
                )
                fig_sim.tight_layout()
                fig_sim.savefig(os.path.join(root,etf,'simulation_perf_lr_{lr}'))
                
            fig2,ax2 = plt.subplots(figsize = (8,6))
            sns.barplot(data=results,x="ETF",y="Final_Value",hue="Metric",ax=ax2,palette='colorblind') 
            sns.despine(ax=ax2)   
            plt.legend(loc=(1.1,1))
            fig2.tight_layout()
            customize_ax(ax2)
            fig2.savefig(os.path.join(root,'return_metrics_lr_{lr}.png'))
            results.to_csv(os.path.join(root,'overall_results.csv'),index=False)

[*********************100%***********************]  1 of 1 completed


read  4779  lines of data for ticker:  XLB


[*********************100%***********************]  1 of 1 completed


read  504  lines of data for ticker:  XLB
On the 46th day, Bought 5.7810151462596835 shares for $500.0
On the 47th day, Bought 5.749770009199632 shares for $499.99999999999994
On the 48th day, Bought 0 shares for $0.0
On the 49th day, Bought 0 shares for $0.0
On the 50th day, Bought 0 shares for $0.0
On the 51th day, Bought 0 shares for $0.0
On the 52th day, Bought 0 shares for $0.0
On the 53th day, Bought 0 shares for $0.0
On the 54th day, Bought 0 shares for $0.0
On the 72th day, sold 11.530785155459316 shares for $984.9596679793348
On the 200th day, Bought 6.688963210702341 shares for $500.0
On the 201th day, Bought 5 shares for $370.4
On the 202th day, Bought 1 shares for $74.27
On the 203th day, Bought 0 shares for $0.0
On the 217th day, Bought 0 shares for $0.0
On the 218th day, Bought 0 shares for $0.0
On the 219th day, Bought 0 shares for $0.0
On the 220th day, Bought 0 shares for $0.0
On the 221th day, Bought 0 shares for $0.0
On the 222th day, Bought 0 shares for $0.0
On the 

[*********************100%***********************]  1 of 1 completed

...

read  1007  lines of data for ticker:  XLU
On the 64th day, Bought 8.260366760284157 shares for $500.0
On the 65th day, Bought 7 shares for $407.68
On the 66th day, Bought 1 shares for $57.28
On the 67th day, Bought 0 shares for $0.0
On the 69th day, Bought 0 shares for $0.0
On the 95th day, sold 16.260366760284157 shares for $988.4676953576738
On the 258th day, Bought 7.8003120124805 shares for $500.0
On the 278th day, sold 7.8003120124805 shares for $473.5569422776912
On the 307th day, Bought 7.689941556444172 shares for $500.0
On the 334th day, sold 7.689941556444172 shares for $504.38326668717326
On the 349th day, Bought 7.6464291176020796 shares for $500.0
On the 360th day, sold 7.6464291176020796 shares for $492.65942804710204
On the 411th day, Bought 7.200460829493088 shares for $500.0
On the 419th day, sold 7.200460829493088 shares for $490.35138248847926
On the 451th day, Bought 7.428316743425939 shares for $500.0
On the 452th day, Bought 6 shares for $401.46
On the 453th day,

[*********************100%***********************]  1 of 1 completed


read  1258  lines of data for ticker:  XLU


[*********************100%***********************]  1 of 1 completed


read  1007  lines of data for ticker:  XLU
On the 70th day, Bought 8.484642796538266 shares for $500.0
On the 74th day, Bought 8.947745168217608 shares for $499.99999999999994
On the 150th day, Bought 0 shares for $0.0
On the 151th day, Bought 0 shares for $0.0
On the 177th day, sold 17.432387964755875 shares for $1026.2446794851783
On the 471th day, Bought 7.489514679448771 shares for $500.0
On the 477th day, sold 7.489514679448771 shares for $512.5074895146795
On the 496th day, Bought 7.055171440666007 shares for $499.99999999999994
On the 516th day, sold 7.055171440666007 shares for $482.0798645407083


In [3]:
os.listdir()

NameError: name 'os' is not defined

In [ ]:
for i in range(32,41):
    df = pd.read_csv(f'runtest_{i}/overall_results.csv')
    print(i,df.groupby('Metric')['Relative_MSE'].mean())

In [ ]:
scaler_x = StandardScaler()
scaler = StandardScaler()
start,end,start_t,end_t = date_list[1]
df_xlp = get_cleaned_df('XLP',start,end)
df_xlp_test = get_cleaned_df('XLP',start_t,end_t)
model,fig_tr,fig_te = get_trained_model(df_xlp,scaler, scaler_x, metric="SD_Squared_Returns",model_type="LSTM")
final_value, money, shares,passive_value,buy_points,sell_points,preds, t_money,p_money = backtest_strategy_mr(df_xlp_test,model,scaler,scaler_x,"SD_Squared_Returns",sell_scale=1.7,buy_scale=1.7)

In [ ]:
sns.lineplot(t_money)